## Preparing the Data

We will demonstrate fine-tuning open source models in order to classify emails into two categories, based on their content.

We will prepare 950 examples to fine-tune on, and use 50 examples to test the performance of our fine-tune. 



In [78]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['raw_prompt','response'])[:1000]
df_train = df[:950]
df_test = df[950:]

Let's first take a look at our data:

In [79]:
df_train['raw_prompt'].iloc[0]

"From: dougb@comm.mot.com (Doug Bank)\nSubject: Re: Info needed for Cleveland tickets\nReply-To: dougb@ecs.comm.mot.com\nOrganization: Motorola Land Mobile Products Sector\nDistribution: usa\nNntp-Posting-Host: 145.1.146.35\nLines: 17\n\nIn article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:\n\n|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.\n|> Does anybody know if the Tribe will be in town on those dates, and\n|> if so, who're they playing and if tickets are available?\n\nThe tribe will be in town from April 16 to the 19th.\nThere are ALWAYS tickets available! (Though they are playing Toronto,\nand many Toronto fans make the trip to Cleveland as it is easier to\nget tickets in Cleveland than in Toronto.  Either way, I seriously\ndoubt they will sell out until the end of the season.)\n\n-- \nDoug Bank                       Private Systems Division\ndougb@ecs.comm.mot.com          Motorola Communications Sect

In [80]:
df_train['response'].value_counts()

response
baseball    498
hockey      452
Name: count, dtype: int64

In [81]:
df_test['response'].value_counts()

response
baseball    25
hockey      25
Name: count, dtype: int64

Since we are training a text generation model, let's do a bit of (extremely basic) prompt engineering to use the model for classification.

In [82]:
def build_prompt(text: str):
    return f"Prompt: {text}\nCategory: "

def prepare_df(df: pd.DataFrame):
    # df['prompt'] = df.apply(lambda row: build_prompt(row['raw_prompt']), axis=1)
    df['prompt'] = df['raw_prompt'].apply(build_prompt)
    df.drop('raw_prompt', axis=1, inplace=True)

In [ ]:
prepare_df(df_train)

In [84]:
df_train.head()

,response,prompt
0,baseball,Prompt: From: dougb@comm.mot.com (Doug Bank)\n...
1,hockey,Prompt: From: gld@cunixb.cc.columbia.edu (Gary...
2,baseball,Prompt: From: rudy@netcom.com (Rudy Wade)\nSub...
3,hockey,Prompt: From: monack@helium.gas.uug.arizona.ed...
4,baseball,Prompt: Subject: Let it be Known\nFrom: <ISSBT...


The data needs to end up in a CSV file that has two columns: `prompt` and `response`, and that is publicly accessible.

In [ ]:
df_train.to_csv("sports_training_dataset.csv")

Behind the scenes, we upload our csv to `s3://scale-demo-datasets/sports/sports_training_dataset.csv`, which maps to a URL of `https://scale-demo-datasets.s3.us-west-2.amazonaws.com/sports/sports_training_dataset.csv`. We make sure this is publicly accessible.

In [ ]:
# We need to set an environment variable SCALE_API_KEY. 
#   Since we're in an ipynb, we can do this. Otherwise, you just need to set the environment variable however you need to
import dotenv
env_file = ".env"
dotenv.load_dotenv(env_file, override=True)



## Fine-Tuning the Model

Next, we create the fine-tune from our training file via the FineTune API.

In [85]:
from llmengine import FineTune, Completion, Model

FineTune.validate_api_key()

In [ ]:


create_fine_tune_response = FineTune.create(
    model="llama-7b",
    training_file="https://scale-demo-datasets.s3.us-west-2.amazonaws.com/sports/sports_training_dataset.csv",
    validation_file=None,
    hyperparameters={},
    suffix="my-first-fine-tune"
)

fine_tune_id = create_fine_tune_response.fine_tune_id




In [ ]:
# Wait for fine tune to complete

import time
while True:
    fine_tune_status = FineTune.retrieve(fine_tune_id).status
    print(fine_tune_status)
    if fine_tune_status == "SUCCESS":
        break
    elif fine_tune_status in ["FAILURE", "CANCELLED"]:
        raise ValueError("Fine Tune failed")
    time.sleep(10)


Assuming you're running this script for the first time, we can get your fine-tune via looking at all the models available.

In [ ]:
all_models = Model.list().model_endpoints

# We want to get just your fine-tuned models.
your_personal_fine_tunes = [model for model in all_models if not model.spec.public_inference]

your_fine_tuned_model = your_personal_fine_tunes[0].name

If you've already created a fine-tune from a previous run, we can also just use that value

In [86]:
# hard-coded value from a previous run of this script
your_fine_tuned_model = "llama-7b.my-first-finetune.2023-07-17-19-44-20"

## Test the Fine-Tune

Next, we run our model on the test dataset via the Completions API. Since we trained using a prompt template, use that prompt template when making predictions.

In [87]:
def get_classification(prompt: str):
    for _ in range(5):
        try:
            response = Completion.create(model_name=your_fine_tuned_model, prompt=build_prompt(prompt), max_new_tokens=2, temperature=0.01)
            return response.outputs[0].text.rstrip("\n")
        except Exception as e:
            print(e)
    return "Error"

In [ ]:
df_test["predicted_response"] = df_test["raw_prompt"].apply(get_classification)

Let's peek at the data and calculate our test accuracy!

In [89]:
df_test.head()

,raw_prompt,response,predicted_response
950,From: tedward@cs.cornell.edu (Edward [Ted] Fis...,baseball,baseball
951,From: smorris@venus.lerc.nasa.gov (Ron Morris ...,hockey,hockey
952,From: shah@pitt.edu (Ravindra S Shah)\nSubject...,hockey,hockey
953,From: timlin@spot.Colorado.EDU (Michael Timlin...,baseball,baseball
954,From: gp2011@andy.bgsu.edu (George Pavlic)\nSu...,hockey,hockey


In [90]:
num_correct = len(df_test[df_test["predicted_response"] == (df_test["response"])])

In [91]:
num_correct / len(df_test)

0.94

In [92]:
df_test[df_test["predicted_response"] != df_test["response"]]

,raw_prompt,response,predicted_response
974,From: maX <maX@maxim.rinaco.msk.su>\nSubject: ...,hockey,
988,From: jca2@cec1.wustl.edu (Joseph Charles Achk...,hockey,NHL
997,From: apland@mala.bc.ca (Ron Apland)\nSubject:...,hockey,
